In [8]:
import nltk
import json
sw = set(nltk.corpus.stopwords.words('english'))

In [9]:
with open('data/stopwords.json', 'w') as f:
    json.dump({'stopwords': list(sw)}, f)

In [9]:
import torch
from torch.utils.mobile_optimizer import optimize_for_mobile
from model import LSTM
from utils import get_and_preprocess

In [10]:
df, v = get_and_preprocess()

In [12]:
model = LSTM(len(v), 32, 48, max_length=30)
model.load_state_dict(torch.load('models/last_model_LSTM.ptl'))

<All keys matched successfully>

In [13]:
example = torch.randint(0, 1000, (1, 30))
model.eval()
model = model.to('cpu')
# cmod = torch.quantization.convert(model)
# scripted_model = torch.jit.script(cmod)
scripted_model = torch.jit.trace(model, example)
opt_model = optimize_for_mobile(scripted_model)
opt_model._save_for_lite_interpreter('models/mobile_model.ptl')

/home/tugsa/miniconda3/envs/py3/lib/python3.10/site-packages/torch/nn/modules/rnn.py:209: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.input_size != input.size(-1):
/home/tugsa/miniconda3/envs/py3/lib/python3.10/site-packages/torch/nn/modules/rnn.py:230: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if hx.size() != expected_hidden_size:
